# Controller for EXPLOR3R

![EXPLOR3R](http://robotsquare.com/wp-content/uploads/2015/10/EXPLOR3Rwithtouchsensor.jpg)

This notebook is an implementation of simple controller for [EXPLOR3R](http://robotsquare.com/2015/10/06/explor3r-building-instructions/). 

Idea is to make robot bump into the wall, drive back a little and turn 90 degrees to the left.

In [2]:
import sys

import rpyc
from dependencies import Injector
from ev3dev import ev3
from ev3dev.ev3 import TouchSensor, InfraredSensor, LargeMotor, Button

Degree = float 

In [3]:
host = "192.168.2.2"

try:
    conn = rpyc.classic.connect(host)
    
    ev3 = conn.modules["ev3dev.ev3"]
except Exception:
    print(sys.exc_info())

In [5]:
class StopCondition:
    
    def __init__(self, touch_sensor: TouchSensor, infra_sensor: InfraredSensor, infra_dist_threshold: int,
                 button: Button):
        self.button = button
        self.touch_sensor = touch_sensor
        self.infra_sensor = infra_sensor
        self.infra_dist_threshold = infra_dist_threshold

    def __call__(self):
        return self.touch_sensor.value() or self.infra_sensor.value() < self.infra_dist_threshold or self.button.enter 
        

In [6]:

class GoStraight:
    
    def __init__(self, left_wheel: LargeMotor, right_wheel: LargeMotor, speed: int):
        self.speed = speed
        self.right_wheel = right_wheel
        self.left_wheel = left_wheel

    def __call__(self):
        self.left_wheel.run_forever(speed_sp = self.speed)
        self.right_wheel.run_forever(speed_sp = self.speed)
        
class GoBack(GoStraight):
    
    def __init__(self, left_wheel: LargeMotor, right_wheel: LargeMotor, speed: int):
        super().__init__(left_wheel, right_wheel, speed)
        self.speed = -speed
        
    def __call__(self):
        self.left_wheel.run_timed(speed_sp = self.speed, time_sp=500)
        self.right_wheel.run_timed(speed_sp = self.speed, time_sp=500)
        
class Stop:
    
    def __init__(self, left_wheel: LargeMotor, right_wheel: LargeMotor):
        self.right_wheel = right_wheel
        self.left_wheel = left_wheel

    def __call__(self):
        self.left_wheel.stop()
        self.right_wheel.stop() 


Робот повернется на $90^{o}$ если колеса будут деть один полный оборот за 1 секунду `speed_sp=360, time_sp=1000` 
    

In [7]:
class Turn:
    
    def __init__(self, left_wheel: LargeMotor, right_wheel: LargeMotor):
        self.right_wheel = right_wheel
        self.left_wheel = left_wheel
    
    def velocity_from_degree(self, degree: Degree, robot_speed: int):
        n_turns = degree / 90
        denominator = robot_speed / 360
        
        speed = n_turns * 360 / denominator
        time = abs(n_turns * 1000 / denominator)
        
        return speed, time
    
    def __call__(self, degree, speed):
        speed_sp, time_sp = self.velocity_from_degree(degree, speed)
        
        self.left_wheel.run_timed(speed_sp=speed_sp, time_sp=time_sp)
        self.right_wheel.run_timed(speed_sp=-speed_sp, time_sp=time_sp)
    


In [8]:
class WalkingRobot:
    def __init__(self, forward: GoStraight, turn: Turn, stop_condition: StopCondition, stop: Stop,
                 go_back: GoBack, degree: float, speed: int, button: Button):
        self.button = button
        self.speed = speed
        self.degree = degree
        self.go_back = go_back
        self.stop_condition = stop_condition
        self.stop = stop
        self.turn = turn
        self.forward = forward
        
    def run(self):
        try:
            while not self.button.enter:
                while not self.stop_condition():
                    self.forward()
                self.stop()
                self.go_back()
                self.turn(self.degree, self.speed)
        except KeyboardInterrupt as e:
            print(e)
            self.stop()
            

In [11]:
class RobotContainer(Injector):
    robot = WalkingRobot
    forward = GoStraight
    turn = Turn
    stop_condition = StopCondition
    stop = Stop
    go_back = GoBack
    speed = 360
    degree = 90
    infra_dist_threshold = 30
    touch_sensor = ev3.TouchSensor('in1')
    infra_sensor = ev3.InfraredSensor('in2')
    left_wheel = ev3.LargeMotor('outA')
    right_wheel = ev3.LargeMotor('outB')
    button = ev3.Button()

In [12]:
RobotContainer.robot.run()
